In [1]:
import torch, torchvision

In [2]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os
import numpy as np
import cv2
import random
from collections import Counter
import json
from matplotlib import pyplot as plt
import pickle
import time


# import some common detectron2 utilities
from detectron2.data.datasets import register_coco_instances
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [14]:
detectron2.__version__

'0.1.3'

In [3]:
f = open('/home/jupyter/main_dataset/train/label1.json') # dale el path del json
json_file_train = json.load(f)

f = open('/home/jupyter/main_dataset/test/label1.json') # dale el path del json
json_file_test = json.load(f)

f = open('/home/jupyter/main_dataset/validation/label1.json') # dale el path del json
json_file_validation = json.load(f)

In [4]:
register_coco_instances("data_train",
                        {},
                        '/home/jupyter/main_dataset/train/label1.json',
                        '/home/jupyter/main_dataset/train/data') 

register_coco_instances("data_test",
                        {},
                        '/home/jupyter/main_dataset/test/label1.json',
                        '/home/jupyter/main_dataset/test/data') 


register_coco_instances("data_val",
                        {},
                        '/home/jupyter/main_dataset/validation/label1.json',
                        '/home/jupyter/main_dataset/validation/data') 
                        


In [5]:
orden_categorias = []
for i in json_file_train['categories']:
    orden_categorias.append(i['name'])

In [6]:
with open('11_categorias/detectron2_order_categories.pckl', 'wb') as handle: #ojo poner la categoria {}
    pickle.dump(orden_categorias, handle)

In [7]:
# Fijamos los hiperparametros.

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")) # Habrá que ver los modelos que existen y cuales se adaptan mejor a la tarea que queremos
cfg.DATASETS.TRAIN = ("data_train",)
cfg.DATASETS.TEST = () # para controlar el overfitting sería ideal separar en train y val, como es una prueba pasamos.
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 2 # El número de batch es a elección nuestra.
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 20000
cfg.SOLVER.STEPS = []
cfg.OUTPUT_DIR = '/home/jupyter/notebook-javi/object_detection/detectron_2/11_categorias'
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(json_file_train['categories']) # Fijamos el numero de categorias. En nuestro ejemplo hay 601, pero realmente solo estamos usando fotos con 91 categorías

cfg.TEST.EVAL_PERIOD = 500

In [8]:
with open('11_categorias/pipeline.pckl', 'wb') as handle:
    pickle.dump(cfg, handle)

In [9]:
def class_frequency(json_file):
    
    categorias = []
    for i in json_file['annotations']:
        categorias.append(i['category_id'])

    print(f'Number of categories: {len(set(categorias))}')
    print(f'frequency: {Counter(categorias)}')

In [10]:
class_frequency(json_file_train)

Number of categories: 11
frequency: Counter({548: 1998, 522: 1513, 90: 1484, 549: 1118, 42: 1043, 342: 998, 73: 836, 568: 699, 6: 447, 495: 394, 370: 209})


In [11]:
class_frequency(json_file_test)

Number of categories: 11
frequency: Counter({90: 1040, 42: 819, 342: 683, 568: 678, 73: 386, 548: 168, 549: 166, 522: 159, 6: 77, 495: 34, 370: 5})


In [12]:
class_frequency(json_file_validation)

Number of categories: 11
frequency: Counter({90: 1045, 568: 681, 42: 418, 342: 232, 73: 103, 522: 100, 548: 65, 549: 46, 6: 17, 495: 13, 370: 4})


In [13]:
# ver si estan igualmente distribuidas los conjuntos.

In [14]:
start_time = time.time()
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) # revisar esto
trainer.train()
print("--- %s seconds ---" % (time.time() - start_time))

[09/14 15:14:24 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

2022-09-14 15:14:24.689508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-14 15:14:24.875944: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-14 15:14:25.482233: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-09-14 15:14:25.482323: W tensorflow/stream_executor/platform/default/dso_l

[09/14 15:14:26 d2.engine.train_loop]: Starting training from iteration 0
[09/14 15:14:39 d2.utils.events]:  eta: 0:53:41  iter: 19  total_loss: 7.241  loss_cls: 6.557  loss_box_reg: 0.569  loss_rpn_cls: 0.027  loss_rpn_loc: 0.017  time: 0.6239  data_time: 0.0162  lr: 0.000005  max_mem: 3710M
[09/14 15:14:52 d2.utils.events]:  eta: 0:53:48  iter: 39  total_loss: 6.992  loss_cls: 6.449  loss_box_reg: 0.359  loss_rpn_cls: 0.035  loss_rpn_loc: 0.010  time: 0.6434  data_time: 0.0041  lr: 0.000010  max_mem: 3710M
[09/14 15:15:06 d2.utils.events]:  eta: 0:54:54  iter: 59  total_loss: 6.826  loss_cls: 6.207  loss_box_reg: 0.543  loss_rpn_cls: 0.038  loss_rpn_loc: 0.016  time: 0.6643  data_time: 0.0043  lr: 0.000015  max_mem: 3710M
[09/14 15:15:20 d2.utils.events]:  eta: 0:54:46  iter: 79  total_loss: 6.559  loss_cls: 5.842  loss_box_reg: 0.659  loss_rpn_cls: 0.023  loss_rpn_loc: 0.013  time: 0.6704  data_time: 0.0044  lr: 0.000020  max_mem: 3710M
[09/14 15:15:33 d2.utils.events]:  eta: 0:54:3

In [13]:
json_file_train['categories']

[{'id': 0, 'name': 'Accordion', 'supercategory': None},
 {'id': 1, 'name': 'Adhesive tape', 'supercategory': None},
 {'id': 2, 'name': 'Aircraft', 'supercategory': None},
 {'id': 3, 'name': 'Airplane', 'supercategory': None},
 {'id': 4, 'name': 'Alarm clock', 'supercategory': None},
 {'id': 5, 'name': 'Alpaca', 'supercategory': None},
 {'id': 6, 'name': 'Ambulance', 'supercategory': None},
 {'id': 7, 'name': 'Animal', 'supercategory': None},
 {'id': 8, 'name': 'Ant', 'supercategory': None},
 {'id': 9, 'name': 'Antelope', 'supercategory': None},
 {'id': 10, 'name': 'Apple', 'supercategory': None},
 {'id': 11, 'name': 'Armadillo', 'supercategory': None},
 {'id': 12, 'name': 'Artichoke', 'supercategory': None},
 {'id': 13, 'name': 'Auto part', 'supercategory': None},
 {'id': 14, 'name': 'Axe', 'supercategory': None},
 {'id': 15, 'name': 'Backpack', 'supercategory': None},
 {'id': 16, 'name': 'Bagel', 'supercategory': None},
 {'id': 17, 'name': 'Baked goods', 'supercategory': None},
 {'id'